# Update Raven Climate

This script is used to update Raven climate input data using ORMGP's [Near Real-time Climate Data Service](https://owrc.github.io/interpolants/sources/climate-data-service.html).  The advantage here is that the data are fully infilled back to 1901 and are updated nightly. Running this script periodically will keep your Raven model up to date.

## Load Python packages and import data

In [16]:
import os
import pandas as pd
from datetime import datetime
import shapefile
from shapely.geometry import shape
from pyproj import Proj

### Set output directory, user settings

In [25]:
outdir = 'output/current_forcing' # directory for datafile output
nyrs = 30 # number of years for a model run. (can go back to 1901)


if not os.path.exists('output'): os.makedirs('output')
if not os.path.exists(outdir): os.makedirs(outdir)

### Load model subbasins

- collect basin centroids

In [10]:
centroids = dict()
sf = shapefile.Reader('../GIS/CH_Subwatershed_select.shp')
prj = Proj('EPSG:3161')
geom = sf.shapes()
attr = sf.records()
for i in range(len(geom)):
    sid = int(attr[i].SubId)
    cyx = shape(geom[i]).centroid
    lng,lat = prj(cyx.x, cyx.y, inverse=True)
    centroids[sid] = (lng,lat)

### ORMGP CDS API function

This function will call the ORGMP Climate Data Service API and convert the data into Raven *Time Series Input file* (.rvt) format.

In [ ]:
def getDailyAPI(fp,lnglat,dtb):
    url = "http://fews.oakridgeswater.ca:8080/dymetp/{}/{}".format(lnglat[1],lnglat[0])
    df = pd.read_json(url)

    df = df[df.Date>=dtb]
    with open(fp,"w") as f:
        f.write(':MultiData\n')
        f.write(' {} {} {}\n'.format(dtb.strftime("%Y-%m-%d %H:%M:%S"), 1, len(df.index)))        
        f.write(' :Parameters  TEMP_MAX  TEMP_MIN    PRECIP\n')
        f.write(' :Units              C         C      mm/d\n')
        for _, row in df.iterrows():
            f.write('            {:>10.2f}{:>10.2f}{:>10.1f}\n'.format(round(row['Tx'],2)+0, round(row['Tn'],2)+0, row['Rf']+row['Sf']))
        f.write(':EndMultiData\n\n')
    return df['Date'].iloc[-1]

Loop through centriods, collect data, write files

In [ ]:
now = datetime.now()
dtb = datetime(now.year-nyrs-1,10,1)
for sid,lnglat in centroids.items():
    dte = getDailyAPI(outdir+'/m{}.rvt'.format(sid),lnglat,dtb)

Copy the following lines that need to replace those in the Raven *Primary Input file* (.rvi):

In [24]:
print(':StartDate {}'.format(dtb.strftime("%Y-%m-%d %H:%M:%S")))
print(':EndDate   {}'.format(dte.strftime("%Y-%m-%d %H:%M:%S")))

:StartDate 1994-10-01 00:00:00
:EndDate   2025-01-20 13:45:22
